In [7]:
import numpy as np
import math
import copy
import torch
LARGEPRIME = 2**61-1
# torch.random.manual_seed(42)

In [4]:
n=100

In [ ]:
np.random.seed(42)
stream = np.random.randint(1,n,size=n)

In [39]:
exactL2 = np.linalg.norm(stream, ord=2)

In [40]:
stream, exactL2

(array([52, 93, 15, 72, 61, 21, 83, 87, 75, 75, 88, 24,  3, 22, 53,  2, 88,
        30, 38,  2, 64, 60, 21, 33, 76, 58, 22, 89, 49, 91, 59, 42, 92, 60,
        80, 15, 62, 62, 47, 62, 51, 55, 64,  3, 51,  7, 21, 73, 39, 18,  4,
        89, 60, 14,  9, 90, 53,  2, 84, 92, 60, 71, 44,  8, 47, 35, 78, 81,
        36, 50,  4,  2,  6, 54,  4, 54, 93, 63, 18, 90, 44, 34, 74, 62, 14,
        95, 48, 15, 72, 78, 87, 62, 40, 85, 80, 82, 53, 24, 26, 89]),
 583.0583161228386)

In [44]:
class CSNorm(object):
    def __init__(self, d, c, r, k=None, device=None):
        self.r = r # num of rows
        self.c = c # num of columns
        self.d = int(d) # vector dimensionality
        if device is None:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            if (not isinstance(device, torch.device) and
                    not ("cuda" in device or device == "cpu")):
                msg = "Expected a valid device, got {}"
                raise ValueError(msg.format(device))
        self.device = device
        self.table = torch.zeros((r, c), device=self.device)

        torch.random.manual_seed(42)
        rand_state = torch.random.get_rng_state()
        self.hashes = torch.randint(0, LARGEPRIME, (self.r, 6),
                               dtype=torch.int64, device="cpu")
        torch.random.set_rng_state(rand_state)
        self.h1 = self.hashes[:,0:1]
        self.h2 = self.hashes[:,1:2]
        self.h3 = self.hashes[:,2:3]
        self.h4 = self.hashes[:,3:4]
        self.h5 = self.hashes[:,4:5]
        self.h6 = self.hashes[:,5:6]
        
        self.norm = torch.zeros(1, dtype=torch.int64, device=self.device)
#         self.topk = torch.zeros((k,2), dtype=torch.int64, device=self.device)        
        
    def accumulateVec(self, vec):
        assert(len(vec.size()) == 1)
        signs = (((self.h1 * vec + self.h2) * vec + self.h3) * vec + self.h4)
        signs = ((signs % LARGEPRIME % 2) * 2 - 1).float()
        signs = signs.to(self.device)
        # computing bucket hashes (2-wise independence)
        buckets = ((self.h5 * vec) + self.h6) % LARGEPRIME % self.c
        buckets = buckets.to(self.device)
        for r in range(self.r):
            bucket = buckets[r,:]
            sign = signs[r,:]
            self.table[r,:] += torch.bincount(input=bucket,
                                                weights=sign,
                                                minlength=self.c)
    def get_norm():
        norms = torch.norm(self.table, p=2, dim=1, keepdim=False, out=None, dtype=torch.int64)
        print(norms.size)
        assert norms.size == self.c
        return torch.mean(norms)
    
#     def _findValues(self, vec):
#         # computing sign hashes (4 wise independence)
#         signs = (((self.h1 * vec + self.h2) * vec + self.h3) * vec + self.h4)
#         signs = ((signs % LARGEPRIME % 2) * 2 - 1).float()
#         signs = signs.to(self.device)

#         # computing bucket hashes (2-wise independence)
#         buckets = ((self.h5 * vec) + self.h6) % LARGEPRIME % self.c
#         buckets = buckets.to(self.device)  
#         # estimating frequency of input coordinates
#         d = vec.size()[0]
#         vals = torch.zeros(self.r, d, device=self.device)
#         for r in range(self.r):
#             vals[r] = self.table[r, buckets[r]] * signs[r]
#         return vals.median(dim=0)[0]     
            
#     def query(self, vec):
#         signs = (((self.h1 * vec + self.h2) * vec + self.h3) * vec + self.h4)
#         signs = ((signs % LARGEPRIME % 2) * 2 - 1).float()
#         signs = signs.to(self.device)
#         # computing bucket hashes (2-wise independence)
#         buckets = ((self.h5 * vec) + self.h6) % LARGEPRIME % self.c
#         buckets = buckets.to(self.device)
#         vals = torch.zeros(self.r, vec.size()[0],dtype=torch.int64, device=self.device)#
#         for r in range(self.r):
#             bucket = buckets[r,:]
#             sign = signs[r,:]           
#             vals[r] = self.table[r, bucket] * sign
#         return vals.median(dim=0)[0]



In [55]:
d=10000
c=2
r=1000
k=None
device='cuda'
streamTr=torch.tensor(stream, dtype=torch.int64)

In [54]:
def get_norm(streamTr, csv):
    d = csv.d
    for ii in range(streamTr.shape[0]//d+1):
        substream=streamTr[ii*d:(ii+1)*d]
        csv.accumulateVec(substream)
    norm = csv.get_norm()
    del csv
    return norm

In [ ]:
def update_sketchs(sketchs, norm):
    if len(sketch) < 3:
        return torch.cat((sketchs, streamTr[:2]), 0)        
    else:
        
        

In [62]:
def get_sketchs(stream, d, c, r, k, device):
    sketchs = torch.tensor([], dtype=torch.int64)
    for i in range(stream):
        streamSeg =stream[:i+1]
        if c is None: c=10*k 
        csv = CSNorm( d, c, r, k, device=device)
        norm = get_norm(streamSeg, csv)
        sketchs = update_sketchs(sketchs, norm)
    return sketchs

In [ ]:
def get_CS_HH(stream,d,c,r,k,device):
    if c is None: c=10*k 
    stream_tr=torch.tensor(stream, dtype=torch.int64)
    csv = CSVec( d, c, r, k, device=device)
#     t0=time.time()
    for ii in range(stream_tr.shape[0]//d+1):
        substream=stream_tr[ii*d:(ii+1)*d]
        csv.accumulateVec(substream)
    norm = csv.get_norm()
    HHs=stream_tr.unique()
    tfreqs=csv.query(HHs).cpu().numpy()
    tfreqs=np.clip(tfreqs,0,None)
    idx=np.argsort(-1*tfreqs)    
    HHfreq=np.vstack((HHs.numpy(),tfreqs))
    t=time.time()-t0
    print('sketch counting time:{:.2f}'.format(t))
    shh,sfreq=HHfreq[:,idx][:,:k]
    return shh,sfreq,t